In [ ]:
model = nn.Sequential(nn.Linear(n_in, n_h),
                     nn.ReLU(),
                     nn.Linear(n_h, n_out),
                     nn.Sigmoid())
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
for epoch in range(50):
    # Forward Propagation
    y_pred = model(x)
    # Compute and print loss
    loss = criterion(y_pred, y)
    print('epoch: ', epoch,' loss: ', loss.item())
    # Zero the gradients
    optimizer.zero_grad()
    
    # perform a backward pass (backpropagation)
    loss.backward()
    
    # Update the parameters
    optimizer.step()

In [ ]:
imgs = load_image('./root/cleaned_data/img_0_22.png')

In [ ]:
import numpy as np
import png
import pydicom
from sklearn.preprocessing import normalize

from os import listdir
from os.path import isfile, join
import os
import torch
import torch.nn as nn

from roi import RoiLearn
from preprocessor import Preprocessor

In [ ]:
roii = RoiLearn()
preprocessor = Preprocessor()

dataLoader = preprocessor.create_datasetLoaderDCM('./root')
preprocessor.show_some_loadedData()

roii.build_model()

roii.propagate_from_dataLoader(dataLoader)
#y_pred = roi.propagate().detach().numpy()
#save_image(y_pred[0,0,:,:]*255, 'pred.png')

In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from dicom_reader import DCMreader

class RoiDataset(Dataset):
    """Roi dataset."""

    def __init__(self, csv_file, root_dir, transform=None, target_transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with contour data.
            root_dir (string): Directory for .dcm images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        'height is  (y length) width is  (x length)'
        self.contour_data = pd.read_csv(csv_file,sep=';', names=('slice', 'frame', 'xmin', 'ymin', 'height','width' ))
        self.dcm_images = DCMreader(root_dir)
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.contour_data)
    
    def __getitem__(self, idx):
        cont = ds.contour_data.iloc[idx]
        sl = int(cont['slice'])
        fr = int(cont['frame'])
        
        mask = np.zeros((224,224))
        mask[ int(cont['ymin']): int(cont['ymin']) + int(cont['height']),int(cont['xmin']): int(cont['xmin']) + int(cont['width'])] = 1
        image = self.dcm_images.get_image(sl,fr)
        
        sample = {'image': image, 'mask': mask}

        if self.transform:
            sample['image'] = self.transform(sample['image'])
        if self.target_transform:
            sample['mask'] = self.target_transform(sample['mask'])

        return sample
        

In [ ]:
from sklearn.preprocessing import StandardScaler
from skimage.transform import resize


class ReScale64(object):
    """Scale down ndarrays in sample to Tensors."""

    def __call__(self, data):
        data = resize(data, (64, 64))
        return data


class ReScale32(object):
    """Scale down ndarrays in sample to Tensors."""

    def __call__(self, data):
        data = resize(data, (32, 32))
        data = np.reshape(data, (1024))
        return data
    

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, data):
        return torch.from_numpy(data)
    
class StandardScale(object):
    """Standard scale ndarrays."""

    def __call__(self, data):
        scaler = StandardScaler()
        data = scaler.fit_transform(data)
        
        # add color axis because
        # numpy image: H x W
        # torch image: C X H X W
        data = np.expand_dims(data, axis=0)
        return data


In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torchvision.transforms import Compose
from roi_dataset import RoiDataset
import transform_classes
from roi import RoiLearn
import torch


csv_file = 'C:/Users/Ede/Documents/Iskola/onw_heart/root/rectangle.csv'
root_dir = 'C:/Users/Ede/Documents/Iskola/onw_heart/root/DCOMS/'
compose1 = Compose([transform_classes.ReScale64(),transform_classes.StandardScale(),transform_classes.ToTensor()])
compose2 = Compose([transform_classes.ReScale32(),transform_classes.ToTensor()])
ds = RoiDataset(csv_file, root_dir, compose1, compose2)

roi = RoiLearn()
roi.build_model()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(roi.model.parameters(), lr=0.001)

dataset_loader = torch.utils.data.DataLoader(ds,batch_size=8, shuffle=True,num_workers=0)

In [4]:
learning_changed = False
for epoch in range(100):
    for i_batch, sample_batched in enumerate(dataset_loader):
        #print(i_batch, sample_batched['image'].size(),sample_batched['mask'].size())

        # Forward Propagation
        y_pred = roi.model(sample_batched['image'])
        # Compute and print loss
        loss = criterion(y_pred, sample_batched['mask'])
        print('epoch: ', epoch,' loss: ', loss.item())
                # Zero the gradients
        optimizer.zero_grad()

        # perform a backward pass (backpropagation)
        loss.backward()

        # Update the parameters
        optimizer.step()

C:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


epoch:  0  loss:  0.08253097283586536
epoch:  0  loss:  0.026261991902712487
epoch:  0  loss:  0.016110081718059506
epoch:  1  loss:  0.012049676664739665
epoch:  1  loss:  0.011122760356945293
epoch:  1  loss:  0.008210549204470601
epoch:  2  loss:  0.007229371201847472
epoch:  2  loss:  0.012158255233043667
epoch:  2  loss:  0.005626429154125917
epoch:  3  loss:  0.008269149023594062
epoch:  3  loss:  0.005766683162003667
epoch:  3  loss:  0.007672260337395062
epoch:  4  loss:  0.008784636414877478
epoch:  4  loss:  0.004289344876109282
epoch:  4  loss:  0.007149378953071775
epoch:  5  loss:  0.0046511020756120555
epoch:  5  loss:  0.008546228045754953
epoch:  5  loss:  0.004009159913408088
epoch:  6  loss:  0.005076484779999988
epoch:  6  loss:  0.005562846333357329
epoch:  6  loss:  0.003992280149061337
epoch:  7  loss:  0.005525553360605247
epoch:  7  loss:  0.002200583746719563
epoch:  7  loss:  0.004353256973290976
epoch:  8  loss:  0.0030114904320407147
epoch:  8  loss:  0.0014

epoch:  67  loss:  0.0006528146802839412
epoch:  67  loss:  2.4829353370998552e-05
epoch:  67  loss:  3.49986938904954e-05
epoch:  68  loss:  0.0003842380936589692
epoch:  68  loss:  4.151009985328297e-05
epoch:  68  loss:  0.0004340539898355443
epoch:  69  loss:  0.0004052485342324581
epoch:  69  loss:  0.00028088741932097845
epoch:  69  loss:  1.2740918812184009e-05
epoch:  70  loss:  0.0002718624459954813
epoch:  70  loss:  2.992787435176352e-05
epoch:  70  loss:  0.000624810281543595
epoch:  71  loss:  2.9954883482211375e-05
epoch:  71  loss:  0.0006401860227190823
epoch:  71  loss:  3.312385423554144e-05
epoch:  72  loss:  2.323973079390357e-05
epoch:  72  loss:  0.0002760830979535875
epoch:  72  loss:  0.0006205507110447396
epoch:  73  loss:  0.0006460089651560115
epoch:  73  loss:  2.4590086997625395e-05
epoch:  73  loss:  2.518226493332697e-05
epoch:  74  loss:  0.00038992608375335985
epoch:  74  loss:  2.4045993574179183e-05
epoch:  74  loss:  0.00043380223318309773
epoch:  75

In [5]:
from PIL import Image
import numpy as np


for i_batch, sample_batched in enumerate(dataset_loader):
            #print(i_batch, sample_batched['image'].size(),sample_batched['mask'].size())

            # Forward Propagation
    y_pred = roi.model(sample_batched['image'])
            
    y_pred = np.reshape(y_pred.detach().numpy()[0], (32,32))
    img = Image.fromarray(y_pred, 'L')
    img.show()
    if i_batch == 2:
        break

C:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
